In [119]:
import os
import pandas as pd
import numpy as np

In [165]:
#vars
list_of_bands = [["PIN2023_475390.sid_Band_1", "red"],
                 ["PIN2023_475390.sid_Band_2","Green"],
                 ["PIN2023_475390.sid_Band_3","Blue"]
                ]
buildings_fc = "Buildings_ExtractRoofForm"
buildings_fld = "BuildingFID"
out_path = "E:/GIS_Lab_Home/NOAA_Proposal_PHD_home/Data/FFE/FFE/color_extraction.gdb"
list_of_table_names = [
    "Red_zonal_building",
    "Green_zonal_building",
    "Blue_zonal_building"
]
alphabet_list = ["A","B","C","D","E","F"]

In [ ]:
#extract r, g, b values to a table based on the mean
for band in list_of_bands:
    out_table = os.path.join(out_path,band[1]+"_zonal_building")
    arcpy.ia.ZonalStatisticsAsTable(
        in_zone_data=buildings_fc,
        zone_field=buildings_fld,
        in_value_raster=band[0],
        out_table=out_table,
        ignore_nodata="DATA",
        statistics_type="MEAN",
        process_as_multidimensional="CURRENT_SLICE",
        percentile_values=[90],
        percentile_interpolation_type="AUTO_DETECT",
        circular_calculation="ARITHMETIC",
        circular_wrap_value=360,
        out_join_layer=None
    )

In [166]:
#join tables
#get building FID
master_array = arcpy.da.FeatureClassToNumPyArray(os.path.join(out_path,list_of_table_names[0]), ["BuildingFID"])
master_array = master_array.astype(str)

# Convert the ESRI table to a NumPy array
for table_names in list_of_table_names:
#get mean values as numpy array
    #get path
    table_to_arr = os.path.join(out_path,table_names)
    #convert table to array
    array = arcpy.da.FeatureClassToNumPyArray(table_to_arr, ["MEAN"])
    
    #round the mean values
    array = array.astype(int)
    
    #get quotients and remainders
    quotients, remainders = divmod(array,16)
    
    #reassign values
    #convert to object arrays so conversion is possible
    remainders = remainders.astype(object)
    quotients = quotients.astype(object)
    #loop through and grab the number from the array and convert it to the letter in the list, 10=A, 11=B
    for i in range(10,16):
        remainders = np.where(remainders == i, alphabet_list[i-10], remainders)
        quotients = np.where(quotients == i, alphabet_list[i-10], quotients)
    #concatenate
    arr = [str(quotients[i]) + str(remainders[i]) for i in range(len(quotients))]
    #convert to object so appendable
    arr = np.array(arr, dtype=object)
    
    #append to buildings ID list
    master_array = np.column_stack((master_array, arr))


#convert to string
master_array = master_array.astype(str)
# Extract the Building IDs (first column)
building_ids = master_array[:, 0]

# Concatenate the remaining columns (except the first column)
concatenated_values = np.core.defchararray.add(master_array[:, 1], master_array[:, 2])
concatenated_values = np.core.defchararray.add(concatenated_values, master_array[:, 3])

# Create the final array by combining the Building ID with the concatenated values
final_array = np.column_stack((building_ids, concatenated_values))

print(final_array)

[['Building_1' 'AFB5B1']
 ['Building_2' '98A29D']
 ['Building_3' '868B7E']
 ...
 ['Building_2073' '67604C']
 ['Building_2074' '817B5C']
 ['Building_2075' '707772']]


In [176]:
#convert back to a table
out_hex_table = table_full_path = os.path.join(out_path,"out_hex_table")

#define dtypes for structured array
dtype = [('BuildingFID', 'U50'), ('Hex', 'U50')]

#convert to structured array
structured_data = np.array([(row[0], row[1]) for row in final_array], dtype=dtype)

#send back as table
arcpy.da.NumPyArrayToTable(structured_data, out_hex_table)

print("Table created and rows inserted successful")


Table created and rows inserted successful
